# Reinforcement learning: Tic Tac Toe et Alpha Zero

## 1/ Jeu de morpion

On va dans un premier temps créer la classe du jeu du morpion

In [28]:
import numpy as np

class Morpion_board():
    def __init__(self):
        self.board = np.zeros([3, 3]).astype(str)
        self.board[self.board == "0.0"] = ' '
        self.player = 1

    def draw_sign(self, move):
        if hasattr(move, "__len__"):
            row, column = move[0], move[1]
        else:
            row = move//3
            column = move%3
        if self.board[row, column] != ' ':
            return ('Invalid position')
        else:
            if self.player == 0:
                self.board[row, column] = 'O'
                self.player = 1
            elif self.player == 1:
                self.board[row, column] = 'X'
                self.player = 0

    def check_winner(self):
        if self.player == 0:
            # on essaye les 6 points qui permettent de gagner (pour le joueur 0)
            if self.board[0, 0] == 'O':
                if (self.board[0, 0] == 'O' and self.board[0, 1] == 'O' and self.board[
                    0, 2] == 'O'):
                    return True
                elif (self.board[0, 0] == 'O' and self.board[1, 0] == 'O' and self.board[
                    2, 0] == 'O'):
                    return True
                elif (self.board[0, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True
            if self.board[1, 1] == 'O':
                if (self.board[0, 1] == 'O' and self.board[1, 1] == 'O' and self.board[
                    2, 1] == 'O'):
                    return True
                elif (self.board[1, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    1, 2] == 'O'):
                    return True
                elif (self.board[2, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    0, 2] == 'O'):
                    return True
            if self.board[2, 2] == 'O':
                if (self.board[0, 2] == 'O' and self.board[1, 2] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True
                elif (self.board[2, 0] == 'O' and self.board[2, 1] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True

        if self.player == 1:
            # on essaye les 6 points qui permettent de gagner (pour le joueur 1)
            if self.board[0, 0] == 'X':
                if (self.board[0, 0] == 'X' and self.board[0, 1] == 'X' and self.board[
                    0, 2] == 'X'):
                    return True
                elif (self.board[0, 0] == 'X' and self.board[1, 0] == 'X' and self.board[
                    2, 0] == 'X'):
                    return True
                elif (self.board[0, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
            if self.board[1, 1] == 'X':
                if (self.board[0, 1] == 'X' and self.board[1, 1] == 'X' and self.board[
                    2, 1] == 'X'):
                    return True
                elif (self.board[1, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    1, 2] == 'X'):
                    return True
                elif (self.board[2, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    0, 2] == 'X'):
                    return True
            if self.board[2, 2] == 'X':
                if (self.board[0, 2] == 'X' and self.board[1, 2] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
                elif (self.board[2, 0] == 'X' and self.board[2, 1] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
        return False

    def actions(self):  # returns all possible moves
        acts = []
        for row in range(3):
            for col in range(3):
                if self.board[row, col] == ' ':
                    acts.append([row, col])
        return acts

    def __repr__(self):
        return(f"""|{self.board[0,0]}|{self.board[0,1]}|{self.board[0,2]}|
|{self.board[1,0]}|{self.board[1,1]}|{self.board[1,2]}|
|{self.board[2,0]}|{self.board[2,1]}|{self.board[2,2]}|""")


On va maintenant vérifier si ce dernier fonctionne.

In [29]:
M = Morpion_board()
print(M)
M.draw_sign([1,1])
print(M)
print(M.actions())
print(M.check_winner())



| | | |
| | | |
| | | |
| | | |
| |X| |
| | | |
[[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
False


## Les 3 éléments d'Alpha Zéro

- Un réseau de neurone adapté au board qui renvoie la policy (distribution en proba des différents mouvements possibles) ainsi que du vainqueur 
- Monte-Carlo Tree Search (MCTS): comme Alpha Chess Zéro, nous avons implémenté la méthode MCTS plutôt que alpha beta.  On utilise la policy pour générer un ensemble de jeu.
- Evaluation du réseau de neurone, basé sur une évaluation du gagnant du match à la prochaine itération

## Le réseau de neurone 

On cherche dans un premier temps à avoir un plateau de jeu intelligible pour notre neural network. On va donc transformer notre plateau de jeu en 1 dimension en trois dimension. 
![baba](img/board_to_neural.png)

On va donc d'abord définir la fonction encode board, pour passer d'un board à un array numpy.

In [30]:
def encode_board(board):
    board_sate = board.board
    encoded = np.zeros([3,3,3]).astype(int)
    encoder_dict = {'O':1, 'X':0}
    for row in range(3):
        for col in range(3):
            if board_sate[row,col] != ' ':
                encoded[row, col, encoder_dict[board_sate[row, col]]] = 1
    if board.player == 1:
        encoded[:,:,2] = 1
    return encoded

On teste si elle fonctionne 

In [34]:
encoded_board = encode_board(M)
print(encoded_board[:,:,0])
print(encoded_board[:,:,1])
print(encoded_board[:,:,2])
print(f"The player that is about to play is {M.player}")

[[0 0 0]
 [0 1 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
The player that is about to play is 0


On va maintenant aussi coder une fonction decode board, pour passer d'un array numpy à un board state.

In [35]:
def decode_board(encoded):
    decoded = np.zeros([3,3]).astype(str)
    decoded[decoded == '0.0'] = ' '
    decoder_dict = {0:'O', 1:'X'}
    for row in range(3):
        for col in range(3):
            for k in range(2):
                if encoded[row,col,k] == 1:
                    decoded[row,col] = decoder_dict[k]
    mboard = Morpion_board()
    mboard.board = decoded
    mboard.player = encoded[0,0,2]
    return mboard

In [36]:
decoded_board = decode_board(encoded_board)
print(decoded_board)

| | | |
| |O| |
| | | |


On retrouve bien notre plateau initial